In [1]:
import numpy as np
import pandas as pd
from vibes.parsers import read_structure
from vibes.helpers.k_grid import d2k, k2d
from vibes.helpers.supercell import find_cubic_cell, make_cubic_supercell, get_cubicness

In [2]:
gan = read_structure('../gan.in')
aln = read_structure('../aln.in')
gao = read_structure('../gao.in')
si = read_structure('../si.in')

In [3]:
help(d2k)

Help on function d2k in module vibes.helpers.k_grid:

d2k(atoms, kptdensity=3.5, even=True)
    [ase.calculators.calculator.kptdensity2monkhorstpack]
    Convert k-point density to Monkhorst-Pack grid size.
    
    atoms: Atoms object
        Contains unit cell and information about boundary conditions.
    kptdensity: float
        Required k-point density.  Default value is 3.5 point per Ang^-1.
    even: bool
        Round up to even numbers.



In [4]:
# compute k from density
i = 0
kptdensity = 3.5
k = 2 * np.pi * np.sqrt((gan.get_reciprocal_cell()[i]**2).sum()) * kptdensity
k_round = 2 * int(np.ceil(k / 2))
k, k_round

(7.956721295269019, 8)

In [5]:
# compute density from k
i = 0
k = [8, 8, 6]
kptdensity = k / (2 * np.pi * np.sqrt((gan.get_reciprocal_cell()**2).sum(axis=1)))
kptdensity, kptdensity.mean()

(array([3.51903742, 3.51903742, 4.96754856]), 4.001874469136553)

In [6]:
# use k2d
k2d(gan, k_grid=[8, 8, 6])

array([3.51903742, 3.51903742, 4.96754856])

In [7]:
sc, sm = make_cubic_supercell(gan, 100)

In [8]:
print(sc.tags)

[]


**Task:** Find supercells with growing size and determine their k_grid by using the function `kptdensity2monkhorstpack` with a fixed k-point density.

# Compute k_grids for Si, GaN, AlN, GaO

In [9]:
def compute_k_grids(atoms, density, n_range=range(10, 200, 20)):
    data = []
    for nn in n_range:
        sc, sm = make_cubic_supercell(atoms, nn)
        k_grid = d2k(sc, density, even=False)
        rho_k = k2d(sc, k_grid)
        data.append({
            'n_target': nn,
            'n_is': len(sc),
            'cub_nes': np.around(get_cubicness(sc.cell), decimals=3),
            'k_grid': k_grid,
            'rho_k': density,
            'rho_k_is': np.around(rho_k.mean(), decimals=2),
            'rho_ks_is': np.around(rho_k, decimals=1)
    })

    df = pd.DataFrame(data)
    return df

In [10]:
rho = 3.5
n_range = range(10, 500, 50)

materials = [si, gan, aln, gao]

dfs = []
for atoms in materials:
    df = compute_k_grids(atoms, rho, n_range)
    dfs.append(df)

**Warning: Cubicness of supercell is 0.624 (0.243)
**-> Sytems: Ga4O6_12_10i, target size 10
**Warning: Cubicness of supercell is 0.696 (0.337)
**-> Sytems: Ga4O6_12_10i, target size 60
**Warning: Cubicness of supercell is 0.783 (0.479)
**-> Sytems: Ga4O6_12_10i, target size 310
**Warning: Cubicness of supercell is 0.783 (0.479)
**-> Sytems: Ga4O6_12_10i, target size 360


In [11]:
print(f'Density = {rho}')
print(f'n_range = {n_range}')
for atoms, df in zip(materials, dfs):
    print()
    print(atoms.sysname)
    print(df)

Density = 3.5
n_range = range(10, 500, 50)

Si2_227_2b
   cub_nes     k_grid  n_is  n_target  rho_k  rho_k_is        rho_ks_is
0    0.829  [3, 5, 5]    12        10    3.5      4.11  [3.9, 4.3, 4.1]
1    1.000  [3, 3, 3]    64        60    3.5      5.19  [5.2, 5.2, 5.2]
2    0.987  [2, 2, 2]   130       110    3.5      4.36  [4.4, 4.3, 4.3]
3    0.886  [2, 2, 2]   180       160    3.5      4.87  [4.3, 5.2, 5.1]
4    1.000  [2, 2, 2]   216       210    3.5      5.19  [5.2, 5.2, 5.2]
5    0.896  [2, 2, 2]   300       260    3.5      5.81  [6.1, 6.1, 5.2]
6    0.993  [2, 2, 2]   350       310    3.5      6.08  [6.1, 6.1, 6.1]
7    0.960  [2, 2, 2]   400       360    3.5      6.39  [6.1, 6.9, 6.1]
8    0.915  [2, 2, 2]   448       410    3.5      6.61  [6.9, 6.1, 6.9]
9    1.000  [2, 2, 2]   512       460    3.5      6.93  [6.9, 6.9, 6.9]

Ga2N2_186_4b
   cub_nes     k_grid  n_is  n_target  rho_k  rho_k_is        rho_ks_is
0    0.927  [5, 5, 5]    12        10    3.5      3.92  [3.8, 3.8, 